# Fallback detection

The goal of this notebook is to find a method to identify whether a prediction is confident enough or not, and if it is not, to decide whether to give the choice among the first 2,3,4 or n most probable answers (partial fallback), or to simply admit that the question was not well understood (full fallback).

A important constraint is to **NOT** have a threshold to set and tweak, or at least, to have a threshold that holds meaning and doesn't need to be adapted to the number of choices or use case.

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
np = pd.np
from itertools import permutations, combinations

## Samples generation

Here, we create a dataset of probabilities distribution with the following constraints:
 1. Probability distributions have a sum of one
 2. Each distribution is sorted from the most to the less probable class
 3. For n dimensions (classes/intents), we want n reference distributions, i.e. distributions for which it is easy to decide whether to fallback or not and how to do it.

In [16]:
dim = 7
sample_size = 50
p = ['p%i' % i for i in xrange(1,dim+1)]
df = pd.DataFrame(np.random.rand(sample_size,1), columns=p[:1])

for i in xrange(1,dim-1):
    df[p[i]] = (1-df[p[:i]].sum(axis=1)).values*np.random.rand(sample_size)

df[p[-1]] = 1 - df[p[:-1]].sum(axis=1)
    
# Sorting each distribution within itself from the highest to the lowest probability (this step is required)
df = df.apply(lambda s: s.sort_values(ascending=False).values, axis=1)

# Sorting the distributions (this one is just a matter of readabilty)
df = df.sort_values(p, ascending=False).reset_index(drop=True)

# The reference distributions will be placed at the end of the dataframe
for i in xrange(1,dim):
    df.loc[sample_size+i-1,p] = np.array([0.999/i]*i + [0.001/(dim-i)]*(dim-i))
df.loc[sample_size+dim-1,p] = np.array([1.]*dim)/dim

In [4]:
df

,p1,p2,p3,p4,p5,p6,p7
0,0.981837,0.008219,0.006559,0.001658,0.000901,0.000511,0.000315
1,0.979137,0.013502,0.003271,0.002551,0.001247,0.000187,0.000105
2,0.970998,0.020759,0.004675,0.002300,0.000601,0.000453,0.000214
3,0.942014,0.056722,0.001228,0.000017,0.000008,0.000008,0.000003
4,0.937435,0.051652,0.010750,0.000089,0.000065,0.000006,0.000003
5,0.929171,0.056889,0.010270,0.003531,0.000065,0.000043,0.000031
6,0.922351,0.023329,0.021359,0.016592,0.009015,0.005050,0.002303
7,0.901035,0.035870,0.025965,0.022688,0.008735,0.003278,0.002429
8,0.899834,0.086489,0.007627,0.003793,0.001886,0.000313,0.000057
9,0.890154,0.051292,0.027980,0.022508,0.006196,0.001270,0.000600


## Entropy sampling

The entropy of a probability distribution tells us how uniform is a distribution of probability. And is defined as:
$$\sum_{i=1}^n p_i\cdot\log_2(p_i)$$ where n is the dimension of the distribution i.e. the number of classes (intents in our case).<br>
A uniform distribution will therefore have an entropy of $\log_2(n)$ and a mono-modal distribution will have a entropy of 0.

A nice property of entropy is that it represent the number of binary questions that needs to be answered in order to classify the sample with full certainty.<br>
For instance, the entropy of the $\begin{pmatrix}0.5 & 0.5\end{pmatrix}$ distribution is $1\ bit$, which means that by asking one binary question, we would know if the correct prediction is the first class or the second one.<br>
In this simple case the question would be: "Is it the first or second class ?".<br>
Of course, the entropy will be a float most of the time, so we choose the take the lower integer part of the entropy as the number of questions.

In practice, instead of asking several binary questions, we would rather ask a single m-ary question ("Is it class one, two or three ?").<br>
The value of m is actually equal to $\left \lfloor{2^E}\right \rfloor$ where E is the entropy. In fact, when the entropy is zero, we should ask a unary question, i.e. no question at all and just pick the most probable class.

In [5]:
def entropy(df):
    return -(np.log2(df)*df).sum(axis=1)

In [6]:
def entropy_sampling(df):
    return np.int64(2**entropy(df))

## Gini index sampling

The gini index mesures the inequalities within a distribution. A perfectly equal (uniform) distribution will have a gini index of 0, whereas a mono modal distribution will have a gini index of 2.

On the contrary of entropy, there is no theoretical way of going from the gini index to a number of questions, however, the gini index has the advantage of being applicable to any kind of distributions, not only probabilities.<br>
Therefore, it is possible to compute the gini index on subsets of the distribution, if the gini index of the sub-distribution is less than 1, we decide that the distribution is ambiguous and that it could be any of the candidates, so we ask the question.

In [7]:
def gini(df, p):
    acc = 0
    for a, b in permutations(p, 2):
        acc += np.abs(df[a] - df[b])
    return acc/(len(p)*(len(p)-1)*df[p].mean(axis=1))

In [8]:
def gini_sampling(df, p, t=1):
    g = pd.DataFrame()
    for i in xrange(2, len(p)+1):
        g[i] = gini(df, p[:i]).values
    return g.apply(lambda s: s[s<t].index[-1] if s.min() < t else 1, axis=1)

In [9]:
df['entropy'] = entropy(df[p])
df['es'] = entropy_sampling(df[p])
df['gini'] = gini(df, p)
df['gs'] = gini_sampling(df, p)
df

,p1,p2,p3,p4,p5,p6,p7,entropy,es,gini,gs
0,0.981837,0.008219,0.006559,0.001658,0.000901,0.000511,0.000315,0.164146,1,1.977094,1
1,0.979137,0.013502,0.003271,0.002551,0.001247,0.000187,0.000105,0.178355,1,1.977167,1
2,0.970998,0.020759,0.004675,0.002300,0.000601,0.000453,0.000214,0.227692,1,1.971359,1
3,0.942014,0.056722,0.001228,0.000017,0.000008,0.000008,0.000003,0.328482,1,1.960453,1
4,0.937435,0.051652,0.010750,0.000089,0.000065,0.000006,0.000003,0.380757,1,1.950848,1
5,0.929171,0.056889,0.010270,0.003531,0.000065,0.000043,0.000031,0.432341,1,1.940879,1
6,0.922351,0.023329,0.021359,0.016592,0.009015,0.005050,0.002303,0.570632,1,1.872697,1
7,0.901035,0.035870,0.025965,0.022688,0.008735,0.003278,0.002429,0.676255,1,1.852154,1
8,0.899834,0.086489,0.007627,0.003793,0.001886,0.000313,0.000057,0.548116,1,1.918284,1
9,0.890154,0.051292,0.027980,0.022508,0.006196,0.001270,0.000600,0.700864,1,1.860328,1


## Generalized Margin sampling

This approach decomposes the distribution on the basis composed of the following vectors:
 - the mono-modal distribution which is unambiguous => we pick the answer with the highest probability
 - the bi-modal distribution => we ask whether the user wants the first or second most probable intent
 - the tri-modal distribution => same thing as above but with the 3 most probable answers
 - and so on up to the uniform distribution => full fallback

Once the decomposition is done for the given distribution, we simply select the basis vector with the highest component.

The decomposition is simply done by doing the matrix product between the inverse of the canonic representation of the basis and the distribution vector. This inverse actually has a very simple expression which an be derived algebrically. We call $M$ the canonic prepresentation of the basis, and $M^{-1}$ its inverse.


$$
\begin{array}{M M^{-1}}
M = \begin{pmatrix}
1 & \frac{1}{2} & \frac{1}{3} & \frac{1}{4} & \cdots & \frac{1}{n} \\
0 & \frac{1}{2} & \frac{1}{3} & \frac{1}{4} & \cdots & \frac{1}{n} \\
0 & 0 & \frac{1}{3} & \frac{1}{4} & \cdots & \frac{1}{n} \\
0 & 0 & 0 & \frac{1}{4} & \cdots & \frac{1}{n} \\
\vdots & \vdots & \vdots & \ddots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & 0 & \frac{1}{n} \\
\end{pmatrix}
&
M^{-1} = \begin{pmatrix}
1 & 0 & 0 & 0 & \cdots & 0 \\
-2 & 2 & 0 & 0 & \cdots & 0 \\
0 & -3 & 3 & 0 & \cdots & 0 \\
0 & 0 & -4 & 4 & \ddots & \vdots\\
\vdots & \vdots & \vdots &\ddots & \ddots & 0 \\
0 & 0 & \cdots & 0 & -n & n \\
\end{pmatrix}
\end{array}
$$


*NB1: In practice the fallback will be activated when there are more than 4 or 5 possible answers*

*NB2: This is actually a generalisation of margin sampling which is used in active learning https://www.cs.cmu.edu/~tom/10701_sp11/recitations/Recitation_13.pdf*

In [10]:
M_inv = np.diag(xrange(1,dim+1))
m = np.roll(M_inv, -1, axis=1)
m[0,:] = 0
M_inv -= m
M_inv

array([[ 1,  0,  0,  0,  0,  0,  0],
       [-2,  2,  0,  0,  0,  0,  0],
       [ 0, -3,  3,  0,  0,  0,  0],
       [ 0,  0, -4,  4,  0,  0,  0],
       [ 0,  0,  0, -5,  5,  0,  0],
       [ 0,  0,  0,  0, -6,  6,  0],
       [ 0,  0,  0,  0,  0, -7,  7]])

In [11]:
df['gms'] = pd.DataFrame(np.dot(df[p].values, M_inv)).apply(pd.Series.argmax, axis=1) + 1
df

,p1,p2,p3,p4,p5,p6,p7,entropy,es,gini,gs,gms
0,0.981837,0.008219,0.006559,0.001658,0.000901,0.000511,0.000315,0.164146,1,1.977094,1,1
1,0.979137,0.013502,0.003271,0.002551,0.001247,0.000187,0.000105,0.178355,1,1.977167,1,1
2,0.970998,0.020759,0.004675,0.002300,0.000601,0.000453,0.000214,0.227692,1,1.971359,1,1
3,0.942014,0.056722,0.001228,0.000017,0.000008,0.000008,0.000003,0.328482,1,1.960453,1,1
4,0.937435,0.051652,0.010750,0.000089,0.000065,0.000006,0.000003,0.380757,1,1.950848,1,1
5,0.929171,0.056889,0.010270,0.003531,0.000065,0.000043,0.000031,0.432341,1,1.940879,1,1
6,0.922351,0.023329,0.021359,0.016592,0.009015,0.005050,0.002303,0.570632,1,1.872697,1,1
7,0.901035,0.035870,0.025965,0.022688,0.008735,0.003278,0.002429,0.676255,1,1.852154,1,1
8,0.899834,0.086489,0.007627,0.003793,0.001886,0.000313,0.000057,0.548116,1,1.918284,1,1
9,0.890154,0.051292,0.027980,0.022508,0.006196,0.001270,0.000600,0.700864,1,1.860328,1,1


## Distance sampling

Here instead of decomposing the distribution against the n-modal distribution basis, we simply compute the distance of the distribution to each vector of the basis and pick the decision of the closest basis vector.

We can chose between many distances:
 - cityblock (norm-1)
 - cosine (actually a pseudo-norm)
 - euclidean (norm-2)
 - chebyshev (norm-infinity)
 - minkowski p-norms
 - etc.

Note that since we are in a finite dimension space, all distances are equivalent, and therefore, the ranks will stay the same, **except** for the cosine distance which is not a actual distance since $ \frac{x,y}{ = 0 \not\Leftrightarrow x = y $.

In [12]:
M = np.transpose([map(lambda x: 1./x if x >= i else 0, xrange(1, dim+1)) for i in xrange(1, dim+1)])
M

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.5       ,  0.5       ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.33333333,  0.33333333,  0.33333333,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.25      ,  0.25      ,  0.25      ,  0.25      ,  0.        ,
         0.        ,  0.        ],
       [ 0.2       ,  0.2       ,  0.2       ,  0.2       ,  0.2       ,
         0.        ,  0.        ],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667,  0.        ],
       [ 0.14285714,  0.14285714,  0.14285714,  0.14285714,  0.14285714,
         0.14285714,  0.14285714]])

In [13]:
from scipy.spatial.distance import cdist

df['ds'] = pd.DataFrame(cdist(df[p], M, 'cityblock')).apply(pd.Series.argmin, axis=1) + 1
df

,p1,p2,p3,p4,p5,p6,p7,entropy,es,gini,gs,gms,ds
0,0.981837,0.008219,0.006559,0.001658,0.000901,0.000511,0.000315,0.164146,1,1.977094,1,1,1
1,0.979137,0.013502,0.003271,0.002551,0.001247,0.000187,0.000105,0.178355,1,1.977167,1,1,1
2,0.970998,0.020759,0.004675,0.002300,0.000601,0.000453,0.000214,0.227692,1,1.971359,1,1,1
3,0.942014,0.056722,0.001228,0.000017,0.000008,0.000008,0.000003,0.328482,1,1.960453,1,1,1
4,0.937435,0.051652,0.010750,0.000089,0.000065,0.000006,0.000003,0.380757,1,1.950848,1,1,1
5,0.929171,0.056889,0.010270,0.003531,0.000065,0.000043,0.000031,0.432341,1,1.940879,1,1,1
6,0.922351,0.023329,0.021359,0.016592,0.009015,0.005050,0.002303,0.570632,1,1.872697,1,1,1
7,0.901035,0.035870,0.025965,0.022688,0.008735,0.003278,0.002429,0.676255,1,1.852154,1,1,1
8,0.899834,0.086489,0.007627,0.003793,0.001886,0.000313,0.000057,0.548116,1,1.918284,1,1,1
9,0.890154,0.051292,0.027980,0.022508,0.006196,0.001270,0.000600,0.700864,1,1.860328,1,1,1


## pseudo-distance sampling

We use same approach as in distance sampling, but with an pseudo-distance, defined as:
$$
\text{d}(\vec{x},\vec{y}) = \left\lVert \vec{x} - \frac{\vec{x} \cdot \vec{y}}{\lVert \vec{y} \rVert}\vec{y}\right\rVert
$$

This pseudo-distance tries to "fit" y to x before taking the euclidean distance.

In [14]:
def pseudo_distance(x, y):
    return np.linalg.norm(x - np.dot(x, y)*y/np.linalg.norm(y))

In [15]:
df["pds"] = pd.DataFrame(cdist(df[p], M, pseudo_distance)).apply(pd.Series.argmin, axis=1)+1
df

,p1,p2,p3,p4,p5,p6,p7,entropy,es,gini,gs,gms,ds,pds
0,0.981837,0.008219,0.006559,0.001658,0.000901,0.000511,0.000315,0.164146,1,1.977094,1,1,1,1
1,0.979137,0.013502,0.003271,0.002551,0.001247,0.000187,0.000105,0.178355,1,1.977167,1,1,1,1
2,0.970998,0.020759,0.004675,0.002300,0.000601,0.000453,0.000214,0.227692,1,1.971359,1,1,1,1
3,0.942014,0.056722,0.001228,0.000017,0.000008,0.000008,0.000003,0.328482,1,1.960453,1,1,1,1
4,0.937435,0.051652,0.010750,0.000089,0.000065,0.000006,0.000003,0.380757,1,1.950848,1,1,1,1
5,0.929171,0.056889,0.010270,0.003531,0.000065,0.000043,0.000031,0.432341,1,1.940879,1,1,1,1
6,0.922351,0.023329,0.021359,0.016592,0.009015,0.005050,0.002303,0.570632,1,1.872697,1,1,1,1
7,0.901035,0.035870,0.025965,0.022688,0.008735,0.003278,0.002429,0.676255,1,1.852154,1,1,1,1
8,0.899834,0.086489,0.007627,0.003793,0.001886,0.000313,0.000057,0.548116,1,1.918284,1,1,1,1
9,0.890154,0.051292,0.027980,0.022508,0.006196,0.001270,0.000600,0.700864,1,1.860328,1,1,1,1


## Conclusion

When we compare our intuitive guesses to each method, it seems that the best fit to our intuition is the pseudo-distance sampling.<br>
For the reference distribution, it seems that all the method give good results except the gini index, which is too cautious and triggers fallback to often.